In [111]:
fileName = 'Equations\Clean\eq1_hr.jpg'
from skimage.color import rgb2gray
import cv2
from scipy.ndimage import interpolation as inter

import numpy as np
import os
os.getcwd()

'C:\\Users\\KINSHUK\\Documents\\GitHub\\LaTeX-Generation-from-Printed-Equation\\actual_code'

In [57]:
def display_image(image,title):
    scaleX = 0.5
    scaleY = 0.5
    scaleDown = cv2.resize(image, None, fx= scaleX, fy= scaleY, interpolation= cv2.INTER_LINEAR)
    cv2.imshow(title, scaleDown)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [58]:
class Segments:
    def __init__(self, image_val, boundingbox,centroids):
        self.image_val=image_val
        self.boundingbox = boundingbox
        self.centroids = centroids

In [112]:
def correct_skew(image, delta=1, limit=10):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated

In [117]:
##deskew 
##input binary image
## output deskewed image
img = cv2.imread(fileName, cv2.IMREAD_COLOR) 
image=np.copy(img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=10, maxLineGap=250)
for line in lines:

    x1, y1, x2, y2 = line[0]

    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

display_image(img,"Lined image")

angle,rotated=correct_skew(image)
if abs(angle)>5:
    rotated=rotated
else:
    rotated=image
    
display_image(rotated,"rotated image")

print(angle)

0


In [ ]:
'''
image = cv2.imread(fileName)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = 255 - gray
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
print(angle)

if abs(angle)>10:
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    display_image(rotated,"Deskewed")
else:
    rotated=image

img = rotated
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=10, maxLineGap=250)
for line in lines:

    x1, y1, x2, y2 = line[0]

    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

display_image(img,"Lined image")
    
image = rotated
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = 255 - gray
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
print(angle)
'''

In [118]:
img = cv2.imread(fileName, 1)
img = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY) #convert to gray scale

# applying Otsu thresholding 
# as an extra flag in binary  
# thresholding      
ret, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY +  cv2.THRESH_OTSU) #otsu thresholding
ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY) #binary image gives 255 0 image
img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  #to get 0,1 image
binary_image=np.copy(img)
display_image(binary_image,"Binary image")


In [47]:
print(rotated)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [119]:
invert_image=np.ones(binary_image.shape)-binary_image
kernel=np.ones((3,3))
erosion_image = cv2.erode(invert_image, kernel,iterations=1) 

display_image(erosion_image,"Post inverting and eroding image ")


In [120]:
cv2.destroyAllWindows()

#input erosion image 
#output segmented character images and centroids, bounding boxes.

image_segement=np.copy(erosion_image)


image_segement=image_segement*255 #scales up the image values as we need a uint8 image for segmentation
image_segement = image_segement.astype(np.uint8) #convert the image to a uint8 from a float 64



contours, hier = cv2.findContours(np.copy(image_segement), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
display_image(image_segement,"Contours")

d=0
segmented_chars=[]
for ctr in contours:
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)
    #getting centroid
    M = cv2.moments(ctr)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    # Getting ROI
    roi = image_segement[y:y+h, x:x+w]
    x,y,w,h = cv2.boundingRect(ctr)
    print(x,y,w,h,cX, cY)
#     
    blah=cv2.rectangle(np.copy(image_segement),(x,y),(x+w,y+h),(255,255,0),10)
    cv2.circle(blah, (cX, cY), 7, (255, 0, 0), -1)

    display_image(blah,'character: %d'%d)
    
    
    segmented_chars.append(Segments(roi,[x,y,w,h],[cX,cY]))
    
    cv2.imwrite('character_%d.png'%d, roi)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    d+=1

462 276 75 109 502 325
601 321 106 4 653 322
601 290 106 5 653 291
767 276 98 76 815 325
880 234 138 116 952 290
1075 255 106 106 1127 307
1238 234 118 116 1304 292


In [55]:
print(segmented_chars[0].image_val)

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ... 255 255   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
